<h2>BBB SCORE</h2>
<p>Author: Gaokeng Xiao</p>
<p>Date: 2021-01-29</p>
<p>Organization: Guangzhou Molcalx Information and Technology Ltd</p>
<p>Web： http://blog.molcalx.com.cn</p>
<p>Five descriptors will be used in BBB score reported by Gupa(2019)<sup>[1]</sup>：</p>
<ul>
   <li>Aro_R：Number of Aromatic Rings</li>
   <li>HA：Number of Heavy Atoms</li>
   <li>MWHBN = HBN/MW<sup>^</sup>2</li>
    <p>where HBN = HBA + HBD， HBA and HBD are Number of Hydrogen Bond Acceptor and Number of Hydrogen Bond Donor, respectively</p>
   <li>TPSA：Topological Polar Surface Area</li>
   <li>pKa：pKa of the molecule</li>
</ul>
<h2>Reference</h2>
<ol>
    <li>Gupta, M., Lee, H. J., Barden, C. J., & Weaver, D. F. (2019). THE BLOOD-BRAIN BARRIER (BBB) SCORE. Journal of Medicinal Chemistry. doi:10.1021/acs.jmedchem.9b01220</li>
</ol>

In [1]:
import rdkit
from rdkit import Chem
print(rdkit.__version__)

2020.03.2


<h2>Calculate descriptors for BBB score</h2>

In [2]:
def BBB_Descriptor(mol):
    """
    Calculate the descriptors for Gupa(2019) BBB Score
    
    Usage: BBB_Descriptor(mol)
    The above mol should be a RDKit molecule. And return a list including HA, MWHBN, ARO_R and TPSA.
    
    Return: [HA, MWHBN, ARO_R, TPSA]
    
    """
    import rdkit
    import rdkit.Chem.rdMolDescriptors as Descriptor
    
    # Calculate NWHBN
    nHBA = Descriptor.CalcNumHBA(mol)
    nHBD = Descriptor.CalcNumHBD(mol)
    HBN = nHBA + nHBD
    MW = round(Descriptor.CalcExactMolWt(mol, onlyHeavy=False),2)
    MWHBN = round(HBN/(MW**0.5),2)
    
    # calcu7late number of the heavy atoms
    n = mol.GetNumAtoms()
    elements = []
    for i in range(n):
        element = m.GetAtoms()[i].GetSymbol()
        elements.append(element)
    nH = elements.count('H')
    HA = n - nH
    
    #Calculate the number of aromatic rings
    aroR = Descriptor.CalcNumAromaticRings(mol)
    
    #
    tpsa = Descriptor.CalcTPSA(mol)
    
    BBB_DESC = [MW,nHBA,nHBD,HBN,MWHBN,HA,aroR,tpsa]
    return(BBB_DESC)

<h2>Calculate BBB SCORE</h2>

In [3]:
def bbbscore(mol,pka):
    """Calculate BBB SCORE"""
    BBB_DESC = BBB_Descriptor(mol)
    HA = float(BBB_DESC[5])
    MWHBN = float(BBB_DESC[4])
    Aro_R = float(BBB_DESC[6])
    TPSA = float(BBB_DESC[7])
    BBB_SCORE = []
    
    # Calculate P value for Aromatic Rings 
    if Aro_R == 0:
        P_ARO_R=0.336367
        
    elif Aro_R == 1:
        P_ARO_R = 0.816016
        
    elif Aro_R == 2:
        P_ARO_R = 1
        
    elif Aro_R == 3:
        P_ARO_R = 0.691115
        
    elif Aro_R == 4:
        P_ARO_R = 0.199399
        
    elif Aro_R > 4:
        P_ARO_R = 0
    
    # Calculate P value for HA
    if HA > 5 and HA <= 45:
        P_HA = (0.0000443*(HA**3) - 0.004556*(HA**2) + 0.12775*HA - 0.463)/0.624231
    else:
        P_HA = 0
    
    
    # Calculate P value for MWHBN
    if MWHBN > 0.05 and MWHBN <= 0.45:
        P_MWHBN = (26.733*(MWHBN**3) - 31.495*(MWHBN**2) + 9.5202*MWHBN - 0.1358)/0.72258
    else:
        P_MWHBN = 0
    
    # Calculate P value for TPSA
    if TPSA > 0 and TPSA <= 120:
        P_TPSA = (-0.0067*TPSA + 0.9598)/0.9598
    else:
        P_TPSA = 0
    
    # Calculate P value for pKa
    pka = float(pka)
    if pka > 3 and pka <= 11:
        P_PKA = (0.00045068*(pka**4) - 0.016331*(pka**3) + 0.18618*(pka**2) - 0.71043*pka + 0.8579)/0.597488
    else:
        P_PKA = 0
    
    BBB_score = P_ARO_R + P_HA + 1.5*P_MWHBN + 2*P_TPSA + 0.5*P_PKA
    
    BBB_SCORE = BBB_DESC
    BBB_DESC.append(pka)
    BBB_SCORE.append(round(BBB_score,2))
    
    return(BBB_SCORE)

In [4]:
# Print the descriptors and BBB score
def BBB_Score_Report(bbb_score):
    print("Number of Aromatic Rings(Aro_R):",bbb_score[6])
    print("Number of Heavy Atoms(HA):",bbb_score[5])
    print("Molecular Weight(MW):",bbb_score[0])
    print("Number of Hydrogen Bond Acceptor (HBA):",bbb_score[1])
    print("Number of Hydrogen Bond Donor(HBD):",bbb_score[2])
    print("MWHBN = HBN/MW^0.5, HBN = HBA + HBD:",bbb_score[4])
    print("Topological Polar Surface Area(TPSA):",bbb_score[7])
    print("pKa:",bbb_score[8])
    print("BBB Score:",bbb_score[9])

<h2>Case study 1：ACETAMINOPHEN</h2>
<ul>
<li>SMILES: CC(=O)Nc1ccc(O)cc1</li>
<li>pKa = 9.89</li>
<li>MW: 151.16</li>
<li>TPSA: 49.33</li>
<li>HA: 11</li>
<li>HBD: 2</li>
<li>HBA: 2</li>
<li>HB: 4</li>
<li>MWHBN:0.33</li>
<li>ARO_R:1</li>
<li>BBB_SCORE: 4.49</li>
</ul>

In [5]:
import rdkit
from rdkit import Chem
m = Chem.MolFromSmiles('CC(=O)Nc1ccc(O)cc1')
bbb_score = bbbscore(m,9.89)
BBB_Score_Report(bbb_score)

Number of Aromatic Rings(Aro_R): 1
Number of Heavy Atoms(HA): 11
Molecular Weight(MW): 151.06
Number of Hydrogen Bond Acceptor (HBA): 2
Number of Hydrogen Bond Donor(HBD): 2
MWHBN = HBN/MW^0.5, HBN = HBA + HBD: 0.33
Topological Polar Surface Area(TPSA): 49.33
pKa: 9.89
BBB Score: 4.43


<h2>Case study 2：CINNARIZINE</h2>
<p>CINNARIZINE is a CNS drug with pKa = 8.1 and has a BBB score value of 5.04</p> 

In [6]:
import rdkit
from rdkit import Chem
m = Chem.MolFromSmiles('N1(CCN(C\C=C\c2ccccc2)CC1)C(c3ccccc3)c4ccccc4')
bbb_score = bbbscore(m,8.1)
BBB_Score_Report(bbb_score)

Number of Aromatic Rings(Aro_R): 3
Number of Heavy Atoms(HA): 28
Molecular Weight(MW): 368.23
Number of Hydrogen Bond Acceptor (HBA): 2
Number of Hydrogen Bond Donor(HBD): 0
MWHBN = HBN/MW^0.5, HBN = HBA + HBD: 0.1
Topological Polar Surface Area(TPSA): 6.48
pKa: 8.1
BBB Score: 5.01
